In [7]:
import os
import pandas as pd
from decimal import Decimal
import numpy as np
from datetime import datetime, timedelta
from dotenv import load_dotenv
import yfinance as yf


import matplotlib.pyplot as plt
import plotly.express as px
import pytz  # Make sure to import pytz for timezone handling
import seaborn as sns


import requests
import csv
import json

import warnings

In [8]:
load_dotenv()

API_KEY = os.getenv("alpha_vantage_api_key")
NASDAQ_DATA_LINK_API_KEY = os.getenv("NASDAQ_DATA_LINK_API_KEY")

In [3]:
# Addtional setting session
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")
pd.options.mode.copy_on_write = True

In [180]:
# Parameters section

alpha_vantage_api_key = API_KEY # FREE TIER API rate limit is 25 requests per day
alpha_vantage_function = {
    'core':[
        'TIME_SERIES_INTRADA'
        ,'TIME_SERIES_DAILY' # this is daily time series quote
        ,'TIME_SERIES_DAILY_ADJUSTED' # this is daily time series adjusted by split/dividend-adjusted
        ,'GLOBAL_QUOTE'
    ]
    ,'fundmental':[
    'INCOME_STATEMENT'
    ,'BALANCE_SHEET' # this is daily time series quote
    ,'CASH_FLOW' # this is daily time series adjusted by split/dividend-adjusted
    ,'EARNINGS'
    ,'EARNINGS_CALENDAR'
]
}

ticker_symbols = [

    # INDEX
    'IVV'
    # ,'QQQ'

    # # energy
    # 'SLB'
    # ,'HAL'
    # ,'OKE'

    # # consumer staples
    # ,'TGT'
    # ,'STZ'
    # ,'DLTR'
    # ,'DG'
    # ,'TSN'
    # ,'MNST'
    # ,'PG'
    # ,'PM'
    # ,'MO'


    # # finanace
    # ,'TRV'
    # ,'CB'
    # ,'BAC'

    # # technology
    ,'NVDA'
    # ,'MSFT'
    # ,'AMZN'

    # holding stock
    ,'TSM'
    ,'ACN'
    ,'DHI'
    ,'STZ'
    ,'GOOG'
    ,'TGT'
]

# Consumer Staples
# ticker_symbols = [
#     'MO',
#     'ADM',
#     # 'BF.B', # DATA ISSUE
#     'BG',
#     'CPB',
#     'CHD',
#     'CLX',
#     'KO',
#     'CL',
#     'CAG',
#     'STZ', # !!
#     'COST',
#     'DG',
#     'DLTR', # !!
#     'EL',
#     'GIS',
#     'HSY',
#     'HRL',
#     'K',
#     'KVUE',
#     'KDP',
#     'KMB',
#     'KHC',
#     # 'KR', # DATA ISSUE
#     'LW',
#     # 'MKC', # DATA ISSUE
#     'TAP',
#     'MDLZ',
#     'MNST',
#     'PEP',
#     'PM',
#     'PG',
#     'SJM',
#     'SYY',
#     'TGT', # !!
#     'TSN', # !!
#     'WBA', # DATA ISSUE
#     'WMT'
# ]

# Energy
# ticker_symbols = [
# 'APA',
#  'BKR', # !! HIGH PE, LOW PROFITABILITY, HIGH FORECAST,  BUT CANNOT BEAT FORECAST, 总结与TRGP类似, 基本面更稳妥, 盈利预期高, 有待考证
#  'CVX',
#  'COP',
#  'CTRA',
#  'DVN',
#  'FANG',
#  'EOG',
#  'EQT',
#  'XOM',
#  'HAL', # !!!! 与SLB类似
#  'HES', # !  HIGH PE, UNDERVALUE, 市场炒作高预期, 2024表现强劲, 已经被超热了
#  'KMI',
#  'MRO',
#  'MPC',
#  'OXY',
#  'OKE', # !!
#  'PSX',
#  'SLB', # !!! A BIT HIGH PE, GOOD ROE, MED PROFITABILITY, STEADY GROWTH FORECAST, AND CAN BEAT FORECAST, 总结基本面良好，与HAL类型相似, 市场炒作热度比HAL更高, 值得研究
#  'TRGP', # !！ HGIH PE, HIGH ROE, RELATIVE MORE DEBT THAN EQUITY, LOW PROFITABILITY, HIGH FORECAST, BUT CANNOT BEAT FORECAST, 总结基本面一般, 但是盈利预期非常高, 有待考证
#  'VLO',
#  'WMB'
#  ]


# Time intelligent parameters
window_days = 90
end_date = datetime.now()
start_date = end_date - timedelta(days=window_days)
earning_calendar = [
    3  # this will return next 1 qtr forecast earning; nowadays the earning calendar only shows the next 1 qtr forecast earning
    ,6  # this will return next 2 qtr forecast earning
    ,12  # this will return next 4 qtr forecast earning
]

PE_yr_range = 6 # this will return x-1 yr PE range

ticker_dict_json = {}
ticker_dict_pd = {}

In [181]:
symbol = 'IVV'

# STOCK SPLIT FACTOR section
url = f'https://www.alphavantage.co/query?function=SPLITS&symbol={symbol}&apikey={alpha_vantage_api_key}'
r = requests.get(url)
data = r.json()

for key, value in data.items():
    if key == 'data':
        if len(value) > 0:
            stock_split_record_df = pd.DataFrame(value)
            stock_split_record_df['split_factor'] = pd.to_numeric(stock_split_record_df['split_factor'], errors='coerce') # change split_factor series to numeric data
            stock_split_record_df['effective_date'] = pd.to_datetime(stock_split_record_df['effective_date'])
        else:
            stock_split_record_df['split_factor'] = 1
            stock_split_record_df['effective_date'] = datetime.today()

# PE TTM Valuation

In [182]:
# Daily quote section
for j, symbol in enumerate(ticker_symbols):


    # STOCK SPLIT FACTOR section
    url = f'https://www.alphavantage.co/query?function=SPLITS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'data':
            if len(value) > 0:
                stock_split_record_df = pd.DataFrame(value)
                stock_split_record_df['split_factor'] = pd.to_numeric(stock_split_record_df['split_factor'], errors='coerce') # change split_factor series to numeric data
                stock_split_record_df['effective_date'] = pd.to_datetime(stock_split_record_df['effective_date'])
            else:
                stock_split_record_df['split_factor'] = 1
                stock_split_record_df['effective_date'] = datetime.today()


    # Daily quote section
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={alpha_vantage_api_key}&outputsize=full'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Time Series (Daily)':


            selected_cols = [
                '4. close'
            ]

            Daily_stock_df = pd.DataFrame(value).transpose()[selected_cols] # tranpose the dataframe and sub select selected cols

            # Rename columns
            Daily_stock_df.rename(
                columns={
                    '4. close': 'stock_price'
                    }
                ,inplace=True
                )
            
            Daily_stock_df["stock_price"] = Daily_stock_df["stock_price"].astype(str).apply(lambda x: float(x))
            Daily_stock_df["stock_price"] = Daily_stock_df["stock_price"].round(2)
            Daily_stock_df.index = pd.to_datetime(Daily_stock_df.index)


    for date_i in Daily_stock_df.index.date:
        for date_j in stock_split_record_df['effective_date'].dt.date:
            if date_i == date_j:

                # stock price to divided the split factor
                Daily_stock_df.loc[Daily_stock_df.index.date < date_j, 'stock_price'] /= (stock_split_record_df['split_factor'][stock_split_record_df['effective_date'].dt.date == date_j].values[0])


    # Monthly quote section
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Monthly Time Series':
            Monthly_stock_df = pd.DataFrame(value)


    Monthly_stock_df = Monthly_stock_df.transpose()
    Monthly_stock_df.index = pd.to_datetime(Monthly_stock_df.index)


    filter_1 = (Monthly_stock_df.index.year.isin(range((datetime.today().year - PE_yr_range) ,datetime.today().year)))
    filter_2 = (Monthly_stock_df.index.month == 12) # month = 12 to get the year end closing price

    selected_cols = [
        '4. close'
    ]

    Monthly_stock_df = Monthly_stock_df[
        filter_1
        & filter_2
    ][selected_cols]

    # Rename columns
    Monthly_stock_df.rename(
        columns={
            '4. close': 'stock_price'
            }
        ,inplace=True
        )

    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].astype(str).apply(lambda x: float(x))
    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].round(2)

    # modify stock price based on stock split
    for year_i in Monthly_stock_df.index.year:
        for year_j in stock_split_record_df['effective_date'].dt.year:
            if year_i == year_j:

                # stock price to divided the split factor
                Monthly_stock_df.loc[Monthly_stock_df.index.year < year_j, 'stock_price'] /= (stock_split_record_df['split_factor'][stock_split_record_df['effective_date'].dt.year == year_j].values[0])



    # Earning section
    # past earnings from alpha vintage API
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'annualEarnings':

            selected_cols = [
                'fiscalDateEnding'
                ,'reportedEPS'
            ]

            annualEPS_df = pd.DataFrame(value) # tranpose the dataframe and sub select selected cols


            annualEPS_df['fiscalDateEnding'] = pd.to_datetime(annualEPS_df['fiscalDateEnding']).dt.year

            annualEPS_df = annualEPS_df[
                annualEPS_df['fiscalDateEnding'].isin(
                    range(
                        (datetime.today().year - 6) 
                        ,datetime.today().year
                            )
                            )
                            ]

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    annualEPS_df[f'{col}'] = annualEPS_df[f'{col}'].astype(str).apply(lambda x: float(x))

                else:
                    continue

            annualEPS_df["PE"] = Monthly_stock_df["stock_price"].values / annualEPS_df['reportedEPS'].values
            annualEPS_df[f"PE_{PE_yr_range-1}yr_avg"] = annualEPS_df["PE"].mean().round(2)
            annualEPS_df[f"PE_{PE_yr_range-1}yr_std"] = np.std(annualEPS_df["PE"]).round(2)
            annualEPS_df[f"PE_{PE_yr_range-1}yr_volatility_+"] = (annualEPS_df[f"PE_{PE_yr_range-1}yr_avg"] + annualEPS_df[f"PE_{PE_yr_range-1}yr_std"]).round(2) # 这个是PE的波动范围上限
            annualEPS_df[f"PE_{PE_yr_range-1}yr_volatility_-"] = (annualEPS_df[f"PE_{PE_yr_range-1}yr_avg"] - annualEPS_df[f"PE_{PE_yr_range-1}yr_std"]).round(2) # 这个是PE的波动范围下限



        if key == 'quarterlyEarnings':

            selected_cols = [
                'reportedDate'
                ,'reportedEPS'
            ]

            qtrEPS_df = pd.DataFrame(value)[selected_cols] # tranpose the dataframe and sub select selected cols
            qtrEPS_df['reportedDate'] = pd.to_datetime(qtrEPS_df['reportedDate'])

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    qtrEPS_df[col] = qtrEPS_df[col].astype(str).apply(lambda x: float(x) if x not in [None, 'None', 'nan', 'NaN'] else float(0))
                else:
                    continue


    # # forecast 1 qtr earnings from alpha vantage API
    # for i in earning_calendar: comment out the for loop in case of future usage, i can be the parameter of {}month
    CSV_URL = f'https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&symbol={symbol}&horizon=12month&apikey={alpha_vantage_api_key}'
    with requests.Session() as s:
        download = s.get(CSV_URL)
        decoded_content = download.content.decode('utf-8')
        cr = csv.reader(decoded_content.splitlines(), delimiter=',')
        my_list = list(cr)

        forecast_earanings_df = pd.DataFrame(
            columns=my_list[0]
            ,data=my_list[1::]
            )
        
        if forecast_earanings_df['estimate'].head(1).values != '':
            latest_projected_EPS = float(forecast_earanings_df['estimate'].head(1).values)
        else:
            latest_projected_EPS = 0


    # forecast 1 year earnings from yf API, forwardPE, PEG
    # yf data
    yf_data = yf.Ticker(symbol).info
    # alpha vintage data
    url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()
    
    for key, value in data.items():
        if key == 'PEGRatio':
            PEG_12month_projected = round(float(value), 2)

        if key == 'ForwardPE':
            PE_12month_projected = round(float(value), 2)

    forecast_earnings_keys = [
        'forwardEps'
    ]


    if not all(key in yf_data.keys() for key in forecast_earnings_keys):
        # Handle the case where one or more keys are missing
        EPS_12month_projected = 0
    else:
        EPS_12month_projected = yf_data['forwardEps'] # 代表了截止下一个日历年结束的EPS, next year forecasted EPS


    # US Treasury section
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=TREASURY_YIELD&interval=daily&maturity=10year&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'data':
            US_T_10yrs_df = pd.DataFrame(value)
            US_T_10yrs_df['value'] = pd.to_numeric(US_T_10yrs_df['value'], errors='coerce') # change dataframe value to numeric data
            US_T_10yrs_YTM = US_T_10yrs_df['value'][0]




    # Consolidated section
    stock_consolidate_df = Daily_stock_df.head(window_days)


    stock_consolidate_df_date = stock_consolidate_df.index
    for i in stock_consolidate_df_date:
                
        # Filter the DataFrame to include only dates(index) less than or equal to the target date
        filtered_qtrEPS_df = qtrEPS_df[qtrEPS_df['reportedDate'] < i]

        # Select the first four rows from the past_qtrs_EPS
        past_4_qtrs_EPS = filtered_qtrEPS_df.head(4) 
        past_3_qtrs_EPS = filtered_qtrEPS_df.head(3)

        # Calculate the sum of the numeric values in the selected rows
        EPS_TTM = past_4_qtrs_EPS['reportedEPS'].values.sum()

        # assign each index row with the EPS_TTM
        stock_consolidate_df.loc[i, "EPS_TTM"] = EPS_TTM

        if i == max(stock_consolidate_df.index):
            EPS_latest_projected = latest_projected_EPS + past_3_qtrs_EPS['reportedEPS'].values.sum()  # This metrics is the past 3 qtrs post EPS + 1 projected EPS
        else:
            continue

        stock_consolidate_df["EPS_latest_projected"] = EPS_latest_projected
    

    # stock's stats
    stock_consolidate_df["Ticker"] = symbol
    stock_consolidate_df["PE_1yr_forward"] = round(PE_12month_projected, 2)
    stock_consolidate_df["PE_TTM"] = (stock_consolidate_df["stock_price"] / stock_consolidate_df["EPS_TTM"]).round(2)
    stock_consolidate_df["PE_TTM_avg"] = stock_consolidate_df["PE_TTM"].mean().round(2)
    stock_consolidate_df["PE_TTM_std"] = np.std(stock_consolidate_df["PE_TTM"]).round(2)
    stock_consolidate_df["PE_TTM_volatility_+"] = (stock_consolidate_df["PE_TTM_avg"] + stock_consolidate_df["PE_TTM_std"]).round(2) # 这个是PE的波动范围上限
    stock_consolidate_df["PE_TTM_volatility_-"] = (stock_consolidate_df["PE_TTM_avg"] - stock_consolidate_df["PE_TTM_std"]).round(2) # 这个是PE的波动范围下限

    stock_consolidate_df["relative_valuation_TTM_+"] = (stock_consolidate_df["PE_TTM_volatility_+"] * stock_consolidate_df["EPS_TTM"]).round(2) # 这个是relative valuation的价格上限
    stock_consolidate_df["relative_valuation_TTM_-"] = (stock_consolidate_df["PE_TTM_volatility_-"] * stock_consolidate_df["EPS_TTM"]).round(2) # 这个是relative valuation的价格下限
    stock_consolidate_df["relative_valuation_TTM_median"] = (np.median([stock_consolidate_df["relative_valuation_TTM_+"][0], stock_consolidate_df["relative_valuation_TTM_-"][0]])).round(2) #这个是根据最新TTM PE估值的价格中位数

    stock_consolidate_df["relative_valuation_nextQuater_projected_+"] = (stock_consolidate_df["PE_TTM_volatility_+"] * stock_consolidate_df["EPS_latest_projected"]).round(2) # 这个是relative valuation的价格上限
    stock_consolidate_df["relative_valuation_nextQuater_projected_-"] = (stock_consolidate_df["PE_TTM_volatility_-"] * stock_consolidate_df["EPS_latest_projected"]).round(2) # 这个是relative valuation的价格下限
    stock_consolidate_df["relative_valuation_nextQuater_projected_median"] = (np.median([stock_consolidate_df["relative_valuation_nextQuater_projected_+"][0], stock_consolidate_df["relative_valuation_nextQuater_projected_-"][0]])).round(2) #这个是根据3 qtrs post EPS + 1 projected EPS 得出PE估值的价格中位数

    stock_consolidate_df[f"{window_days}_price_min"] = stock_consolidate_df["stock_price"].min().round(2)
    stock_consolidate_df[f"{window_days}_price_max"] = stock_consolidate_df["stock_price"].max().round(2)
    stock_consolidate_df[f"{window_days}_price_avg"] = stock_consolidate_df["stock_price"].mean().round(2)
    stock_consolidate_df[f"{window_days}_price_std"] = np.std(stock_consolidate_df["stock_price"]).round(2)

    stock_consolidate_df[f"PE_{PE_yr_range-1}yr_avg"] = annualEPS_df[f"PE_{PE_yr_range-1}yr_avg"].values[0]
    stock_consolidate_df[f"PE_{PE_yr_range-1}yr_std"] = annualEPS_df[f"PE_{PE_yr_range-1}yr_std"].values[0]
    stock_consolidate_df[f"PE_{PE_yr_range-1}yr_volatility_+"] = annualEPS_df[f"PE_{PE_yr_range-1}yr_volatility_+"].values[0]
    stock_consolidate_df[f"PE_{PE_yr_range-1}yr_volatility_-"] = annualEPS_df[f"PE_{PE_yr_range-1}yr_volatility_-"].values[0]


    stock_consolidate_df["relative_valuation_nextYear_projected_+"] = (stock_consolidate_df["PE_TTM_volatility_+"] * EPS_12month_projected).round(2) # 这个是relative valuation的价格上限
    stock_consolidate_df["relative_valuation_nextYear_projected_-"] = (stock_consolidate_df["PE_TTM_volatility_-"] * EPS_12month_projected).round(2) # 这个是relative valuation的价格下限
    stock_consolidate_df["relative_valuation_nextYear_projected_median"] = (np.median([stock_consolidate_df["relative_valuation_nextYear_projected_+"][0], stock_consolidate_df["relative_valuation_nextYear_projected_-"][0]])).round(2) #这个是根据next year projected EPS 得出PE估值的价格中位数

    stock_consolidate_df["next12months_PEG"] = PEG_12month_projected
    stock_consolidate_df["TTM_PEG"] = (stock_consolidate_df["PE_TTM"] / (((EPS_12month_projected - stock_consolidate_df["EPS_TTM"]) / stock_consolidate_df["EPS_TTM"]) * 100)).round(2) # 这个是截止下一年的EPS growth rate所得出的PEG ratio, <1是undervalue的表现
    
    stock_consolidate_df["nextYear_EPS_growthRate"] = (((EPS_12month_projected - stock_consolidate_df["EPS_TTM"]) / stock_consolidate_df["EPS_TTM"]) * 100).round(2)
    stock_consolidate_df["nextQuater_EPS_growthRate"] = (((stock_consolidate_df["EPS_latest_projected"] - stock_consolidate_df["EPS_TTM"]) / stock_consolidate_df["EPS_TTM"]) * 100).round(2)

    stock_consolidate_df["EarningYield_TTM"] = ((stock_consolidate_df["EPS_TTM"] / stock_consolidate_df["stock_price"]) * 100).round(2)
    stock_consolidate_df["ERP_TTM"] = stock_consolidate_df["EarningYield_TTM"] - US_T_10yrs_YTM




    # filter conditions
    conditions = [
    (stock_consolidate_df["stock_price"] < stock_consolidate_df["relative_valuation_TTM_-"]),
    (stock_consolidate_df["stock_price"] > stock_consolidate_df["relative_valuation_TTM_+"]),
    ((stock_consolidate_df["stock_price"] >= stock_consolidate_df["relative_valuation_TTM_-"]) & (stock_consolidate_df["stock_price"] <= stock_consolidate_df["relative_valuation_TTM_+"])),
    ]

    categories = [
        'undervalued'
        ,'overvalued'
        ,'fair'
        ]

    # This KPI assess if the current stock price is under/over/fair to the current relative valuation
    stock_consolidate_df["curr_assessment"] = None

    for condition, category in zip(conditions, categories):
        stock_consolidate_df.loc[condition, "price_valuation_assessment"] = category





    # Append key-value pairs to the dictionary
    selected_cols = [
    "Ticker"
    ,"stock_price"
    ,"EPS_TTM"
    ,"EPS_latest_projected"
    ,"PE_1yr_forward"
    ,"PE_TTM"
    ,"PE_TTM_avg"
    ,f"PE_{PE_yr_range-1}yr_avg"
    ,f"PE_{PE_yr_range-1}yr_volatility_+"
    ,f"PE_{PE_yr_range-1}yr_volatility_-"
    ,"relative_valuation_TTM_+"
    ,"relative_valuation_TTM_-"
    ,"relative_valuation_TTM_median"
    ,"relative_valuation_nextQuater_projected_+"
    ,"relative_valuation_nextQuater_projected_-"
    ,"relative_valuation_nextQuater_projected_median"
    ,"relative_valuation_nextYear_projected_+"
    ,"relative_valuation_nextYear_projected_-"
    ,"relative_valuation_nextYear_projected_median"
    ,"price_valuation_assessment"
    ,"nextQuater_EPS_growthRate"
    ,"nextYear_EPS_growthRate"
    ,"next12months_PEG"
    ,"TTM_PEG"
    ,"EarningYield_TTM"
    ,"ERP_TTM"
    ]


    # store each stock info as pd into dictionary
    ticker_dict_pd[symbol] = stock_consolidate_df[selected_cols]
    # transfer pandas dataframe to json format, and each stock info into dictionary
    ticker_dict_json[symbol] = stock_consolidate_df[selected_cols].to_json()

    if j == 0:
        # screener df creation
        # screener df will store each stock's consolidate df's first row and union them together for screening purposee
        ticker_screen_df = pd.DataFrame(
            columns=selected_cols
        ) 
        stock_consolidate_df_values = stock_consolidate_df[selected_cols].values[0]

        # Insert rows into the DataFrame
        ticker_screen_df.loc[j] = stock_consolidate_df_values

    else:
        stock_consolidate_df_values = stock_consolidate_df[selected_cols].values[0]
        ticker_screen_df.loc[j] = stock_consolidate_df_values

In [183]:
ticker_dict_pd['NVDA']

,Ticker,stock_price,EPS_TTM,EPS_latest_projected,PE_1yr_forward,PE_TTM,PE_TTM_avg,PE_5yr_avg,PE_5yr_volatility_+,PE_5yr_volatility_-,relative_valuation_TTM_+,relative_valuation_TTM_-,relative_valuation_TTM_median,relative_valuation_nextQuater_projected_+,relative_valuation_nextQuater_projected_-,relative_valuation_nextQuater_projected_median,relative_valuation_nextYear_projected_+,relative_valuation_nextYear_projected_-,relative_valuation_nextYear_projected_median,price_valuation_assessment,nextQuater_EPS_growthRate,nextYear_EPS_growthRate,next12months_PEG,TTM_PEG,EarningYield_TTM,ERP_TTM
2024-08-12,NVDA,109.020,1.800,2.16,40.0,60.57,66.75,755.51,1219.27,291.75,128.59,111.71,120.15,154.31,134.05,144.18,267.9,232.73,250.32,undervalued,20.00,108.33,1.19,0.56,1.65,-2.29
2024-08-09,NVDA,104.750,1.800,2.16,40.0,58.19,66.75,755.51,1219.27,291.75,128.59,111.71,120.15,154.31,134.05,144.18,267.9,232.73,250.32,undervalued,20.00,108.33,1.19,0.54,1.72,-2.22
2024-08-08,NVDA,104.970,1.800,2.16,40.0,58.32,66.75,755.51,1219.27,291.75,128.59,111.71,120.15,154.31,134.05,144.18,267.9,232.73,250.32,undervalued,20.00,108.33,1.19,0.54,1.71,-2.23
2024-08-07,NVDA,98.910,1.800,2.16,40.0,54.95,66.75,755.51,1219.27,291.75,128.59,111.71,120.15,154.31,134.05,144.18,267.9,232.73,250.32,undervalued,20.00,108.33,1.19,0.51,1.82,-2.12
2024-08-06,NVDA,104.250,1.800,2.16,40.0,57.92,66.75,755.51,1219.27,291.75,128.59,111.71,120.15,154.31,134.05,144.18,267.9,232.73,250.32,undervalued,20.00,108.33,1.19,0.53,1.73,-2.21
2024-08-05,NVDA,100.450,1.800,2.16,40.0,55.81,66.75,755.51,1219.27,291.75,128.59,111.71,120.15,154.31,134.05,144.18,267.9,232.73,250.32,undervalued,20.00,108.33,1.19,0.52,1.79,-2.15
2024-08-02,NVDA,107.270,1.800,2.16,40.0,59.59,66.75,755.51,1219.27,291.75,128.59,111.71,120.15,154.31,134.05,144.18,267.9,232.73,250.32,undervalued,20.00,108.33,1.19,0.55,1.68,-2.26
2024-08-01,NVDA,109.210,1.800,2.16,40.0,60.67,66.75,755.51,1219.27,291.75,128.59,111.71,120.15,154.31,134.05,144.18,267.9,232.73,250.32,undervalued,20.00,108.33,1.19,0.56,1.65,-2.29
2024-07-31,NVDA,117.020,1.800,2.16,40.0,65.01,66.75,755.51,1219.27,291.75,128.59,111.71,120.15,154.31,134.05,144.18,267.9,232.73,250.32,fair,20.00,108.33,1.19,0.60,1.54,-2.40
2024-07-30,NVDA,103.730,1.800,2.16,40.0,57.63,66.75,755.51,1219.27,291.75,128.59,111.71,120.15,154.31,134.05,144.18,267.9,232.73,250.32,undervalued,20.00,108.33,1.19,0.53,1.74,-2.20


In [184]:
ticker_screen_df.sort_values(by=['ERP_TTM']
                             ,ascending=False)

,Ticker,stock_price,EPS_TTM,EPS_latest_projected,PE_1yr_forward,PE_TTM,PE_TTM_avg,PE_5yr_avg,PE_5yr_volatility_+,PE_5yr_volatility_-,relative_valuation_TTM_+,relative_valuation_TTM_-,relative_valuation_TTM_median,relative_valuation_nextQuater_projected_+,relative_valuation_nextQuater_projected_-,relative_valuation_nextQuater_projected_median,relative_valuation_nextYear_projected_+,relative_valuation_nextYear_projected_-,relative_valuation_nextYear_projected_median,price_valuation_assessment,nextQuater_EPS_growthRate,nextYear_EPS_growthRate,next12months_PEG,TTM_PEG,EarningYield_TTM,ERP_TTM
4,DHI,172.07,14.89,14.61,10.93,11.56,10.38,9.61,11.87,7.35,167.07,142.05,154.56,163.92,139.38,151.65,178.85,152.07,165.46,overvalued,-1.88,7.05,0.57,1.64,8.65,4.71
7,TGT,135.02,8.91,9.3,14.58,15.15,17.14,20.75,26.79,14.71,162.43,143.01,152.72,169.54,149.26,159.4,189.41,166.76,178.08,undervalued,4.38,16.61,2.08,0.91,6.6,2.66
5,STZ,238.8,12.72,13.14,17.73,18.77,20.76,22.25,24.47,20.03,277.17,250.97,264.07,286.32,259.25,272.78,330.99,299.7,315.35,undervalued,3.3,19.42,1.53,0.97,5.33,1.39
6,GOOG,163.95,6.97,7.26,21.74,23.52,26.81,25.05,28.23,21.87,197.25,176.48,186.86,205.46,183.82,194.64,245.93,220.03,232.98,undervalued,4.16,24.68,1.21,0.95,4.25,0.31
3,ACN,313.96,11.88,11.95,24.33,26.43,25.94,30.81,38.93,22.69,322.3,294.03,308.16,324.2,295.76,309.98,346.45,316.06,331.26,fair,0.59,7.49,2.05,3.53,3.78,-0.16
2,TSM,167.63,5.6,6.09,25.97,29.93,29.75,22.66,30.05,15.27,183.34,149.86,166.6,199.39,162.97,181.18,270.43,221.04,245.74,fair,8.75,47.5,1.46,0.63,3.34,-0.6
0,IVV,536.11,8.91,7.11,14.58,60.17,59.84,20.75,26.79,14.71,551.35,515.0,533.17,439.97,410.96,425.46,0.0,0.0,0.0,fair,-20.2,-100.0,2.08,-0.6,1.66,-2.28
1,NVDA,109.02,1.8,2.16,40.0,60.57,66.75,755.51,1219.27,291.75,128.59,111.71,120.15,154.31,134.05,144.18,267.9,232.73,250.32,undervalued,20.0,108.33,1.19,0.56,1.65,-2.29


In [68]:
import requests
ticker_symbols = ['AAPL']
for symbol in ticker_symbols:
    print(symbol)

    # COMPANY OVERVIEW
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    
    for key, value in data.items():
        # print(key)
        if key  == 'MarketCapitalization':
            stock_mkt_cap = float(value)



    
    # INCOME STATEMENT
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()


    for key, value in data.items():
        if key == 'annualReports':
            annual_income_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_income_df = annual_income_df.sort_values('fiscalDateEnding', ascending=True)

            # annual measurements cols
            annual_income_metric_cols = annual_income_df.columns[1:]

            # Calculate quarter-over-quarter change percentage
            for column in annual_income_metric_cols:  # Exclude the 'fiscalDateEnding' column
                annual_income_df[column] = pd.to_numeric(annual_income_df[column], errors='coerce')
                annual_income_df[f'{column}_YoY'] = annual_income_df[column].pct_change() * 100 
            
            annual_income_YoY_metric_cols = [col for col in annual_income_df.columns if 'YoY' in col]

            # ratios of income statement calculation
            annual_income_df['gross_margin_%'] = (annual_income_df['grossProfit'] / annual_income_df['totalRevenue']) * 100
            annual_income_df['operating_margin_%'] = (annual_income_df['operatingIncome'] / annual_income_df['totalRevenue']) * 100
            annual_income_df['net_margin_%'] = (annual_income_df['netIncome'] / annual_income_df['totalRevenue']) * 100



        if key == 'quarterlyReports':
            qtr_income_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_income_df = qtr_income_df.sort_values('fiscalDateEnding', ascending=True)

            # qtr measurements cols
            qtr_income_metric_cols = qtr_income_df.columns[1:]
            
            # Calculate quarter-over-quarter change percentage
            for column in qtr_income_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_income_df[column] = pd.to_numeric(qtr_income_df[column], errors='coerce')
                qtr_income_df[f'{column}_QoQ'] = qtr_income_df[column].pct_change() * 100

            qtr_income_QoQ_metric_cols = [col for col in qtr_income_df.columns if 'QoQ' in col]

            # ratios of income statement calculation
            qtr_income_df['gross_margin_%'] = (qtr_income_df['grossProfit'] / qtr_income_df['totalRevenue']) * 100
            qtr_income_df['operating_margin_%'] = (qtr_income_df['operatingIncome'] / qtr_income_df['totalRevenue']) * 100
            qtr_income_df['net_margin_%'] = (qtr_income_df['netIncome'] / qtr_income_df['totalRevenue']) * 100

    income_ratio_cols = [
        'gross_margin_%'
        ,'operating_margin_%'
        ,'net_margin_%'
        ,'netIncome'
        ]
    
    
    # BALANCESHEET
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        print(key, value)

        if key == 'annualReports':
            annual_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_balancesheet_df = annual_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

            for column in annual_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                annual_balancesheet_df[column] = pd.to_numeric(annual_balancesheet_df[column], errors='coerce')

            annual_balancesheet_df['current_ratio'] = (annual_balancesheet_df['totalCurrentAssets'] / annual_balancesheet_df['totalCurrentLiabilities'])
            annual_balancesheet_df['working_capital'] = annual_balancesheet_df['totalCurrentAssets'] - annual_balancesheet_df['totalCurrentLiabilities']
            annual_balancesheet_df['longTermDebt_to_workingCp_ratio'] = (annual_balancesheet_df['longTermDebt'] / annual_balancesheet_df['working_capital'])
            annual_balancesheet_df['debtEquity_ratio'] = (annual_balancesheet_df['totalLiabilities'] / annual_balancesheet_df['totalShareholderEquity'])
            annual_balancesheet_df['quick_ratio'] = ((annual_balancesheet_df['totalCurrentAssets'] - annual_balancesheet_df['inventory']) / annual_balancesheet_df['totalCurrentLiabilities'])

            
        if key == 'quarterlyReports':
            qtr_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_balancesheet_df = qtr_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

            for column in qtr_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_balancesheet_df[column] = pd.to_numeric(qtr_balancesheet_df[column], errors='coerce')

            qtr_balancesheet_df['current_ratio'] = (qtr_balancesheet_df['totalCurrentAssets'] / qtr_balancesheet_df['totalCurrentLiabilities'])
            qtr_balancesheet_df['working_capital'] = qtr_balancesheet_df['totalCurrentAssets'] - qtr_balancesheet_df['totalCurrentLiabilities']
            qtr_balancesheet_df['longTermDebt_to_workingCp_ratio'] = (qtr_balancesheet_df['longTermDebt'] / qtr_balancesheet_df['working_capital'])
            qtr_balancesheet_df['debtEquity_ratio'] = (qtr_balancesheet_df['totalLiabilities'] / qtr_balancesheet_df['totalShareholderEquity'])
            qtr_balancesheet_df['quick_ratio'] = ((qtr_balancesheet_df['totalCurrentAssets'] - qtr_balancesheet_df['inventory']) / qtr_balancesheet_df['totalCurrentLiabilities'])


    balancesheet_ratio_cols = [
        'current_ratio'
        ,'working_capital'
        ,'longTermDebt_to_workingCp_ratio'
        ,'debtEquity_ratio'
        ,'quick_ratio'
        ,'totalShareholderEquity'
    ]


    stock_ratios_annual_consolidate_df = pd.DataFrame()
    stock_ratios_qtr_consolidate_df = pd.DataFrame()

    stock_ratios_annual_consolidate_df[income_ratio_cols] = annual_income_df[income_ratio_cols]
    stock_ratios_annual_consolidate_df[balancesheet_ratio_cols] = annual_balancesheet_df[balancesheet_ratio_cols]

    stock_ratios_qtr_consolidate_df[income_ratio_cols] = qtr_income_df[income_ratio_cols]
    stock_ratios_qtr_consolidate_df[balancesheet_ratio_cols] = qtr_balancesheet_df[balancesheet_ratio_cols]

    # calculating new consolidated metrics 
    # annual df
    stock_ratios_annual_consolidate_df['ROE_%'] = (annual_income_df['netIncome'] / annual_balancesheet_df['totalShareholderEquity']) * 100
    stock_ratios_annual_consolidate_df['liquidation_value'] = ((annual_balancesheet_df['totalAssets'] - annual_balancesheet_df['intangibleAssets']) - annual_balancesheet_df['totalLiabilities'])
    stock_ratios_annual_consolidate_df['liquidation_mktcap_ratio_%'] = round((stock_ratios_annual_consolidate_df['liquidation_value'] / stock_mkt_cap) * 100, 2)
    stock_ratios_annual_consolidate_df['fiscalDateEnding'] = annual_balancesheet_df['fiscalDateEnding']

    # qtr df
    stock_ratios_qtr_consolidate_df['ROE_%'] = (qtr_income_df['netIncome'] / qtr_balancesheet_df['totalShareholderEquity']) * 100
    stock_ratios_qtr_consolidate_df['liquidation_value'] = ((qtr_balancesheet_df['totalAssets'] - qtr_balancesheet_df['intangibleAssets']) - qtr_balancesheet_df['totalLiabilities'])
    stock_ratios_qtr_consolidate_df['liquidation_mktcap_ratio_%'] = round((stock_ratios_qtr_consolidate_df['liquidation_value'] / stock_mkt_cap) * 100, 2)
    stock_ratios_qtr_consolidate_df['fiscalDateEnding'] = qtr_balancesheet_df['fiscalDateEnding']
    

AAPL
symbol AAPL
annualReports [{'fiscalDateEnding': '2023-09-30', 'reportedCurrency': 'USD', 'totalAssets': '352583000000', 'totalCurrentAssets': '143566000000', 'cashAndCashEquivalentsAtCarryingValue': '29965000000', 'cashAndShortTermInvestments': '61555000000', 'inventory': '6331000000', 'currentNetReceivables': '60985000000', 'totalNonCurrentAssets': '209017000000', 'propertyPlantEquipment': '43715000000', 'accumulatedDepreciationAmortizationPPE': '70884000000', 'intangibleAssets': 'None', 'intangibleAssetsExcludingGoodwill': 'None', 'goodwill': 'None', 'investments': '264965000000', 'longTermInvestments': '100544000000', 'shortTermInvestments': '31590000000', 'otherCurrentAssets': '14695000000', 'otherNonCurrentAssets': '64758000000', 'totalLiabilities': '290437000000', 'totalCurrentLiabilities': '145308000000', 'currentAccountsPayable': '62611000000', 'deferredRevenue': '20161000000', 'currentDebt': '15972000000', 'shortTermDebt': '5985000000', 'totalNonCurrentLiabilities': '1451

In [67]:
stock_ratios_annual_consolidate_df

,gross_margin_%,operating_margin_%,net_margin_%,netIncome,current_ratio,working_capital,longTermDebt_to_workingCp_ratio,debtEquity_ratio,quick_ratio,totalShareholderEquity,ROE_%,liquidation_value,liquidation_mktcap_ratio_%,fiscalDateEnding
14,40.139844,27.362778,19.193567,8235000000,2.742482,20049000000,NaN,0.501296,2.702938,31640000000,26.027181,3.118700e+10,0.95,2009-09-30
13,39.377539,28.187045,21.484094,14013000000,2.011292,20956000000,NaN,0.573162,1.960573,47791000000,29.321420,4.670800e+10,1.42,2010-09-30
12,40.478896,31.215069,23.946642,25922000000,1.608438,17018000000,NaN,0.518906,1.580694,76615000000,33.834106,7.218300e+10,2.20,2011-09-30
11,43.871240,35.295959,26.665091,41733000000,1.495849,19111000000,0.000000,0.489417,1.475326,118210000000,35.304120,1.128510e+11,3.43,2012-09-30
10,37.654444,28.692307,21.687728,37037000000,1.678639,29628000000,0.572431,0.675449,1.638234,123549000000,29.977580,1.177930e+11,3.58,2013-09-30
9,38.669269,28.782804,21.659878,39510000000,1.080113,5083000000,5.702735,1.078397,1.046842,111547000000,35.420047,1.027890e+11,3.13,2014-09-30
8,40.185937,30.573177,22.917650,53394000000,1.108771,8768000000,6.546533,1.432617,1.079630,119355000000,44.735453,1.103460e+11,3.36,2015-09-30
7,39.075956,27.835410,21.186798,45687000000,1.352669,27863000000,3.057819,1.508292,1.325684,128249000000,35.623670,1.196290e+11,3.64,2016-09-30
6,38.469860,26.760428,21.092421,48351000000,1.276063,27831000000,4.045704,1.799906,1.227905,134047000000,36.070184,1.260320e+11,3.83,2017-09-30
5,38.343719,26.694027,22.414202,59531000000,1.132926,15410000000,7.266970,2.413301,1.098802,107147000000,55.560118,NaN,NaN,2018-09-30


In [69]:
stock_ratios_qtr_consolidate_df

,gross_margin_%,operating_margin_%,net_margin_%,netIncome,current_ratio,working_capital,longTermDebt_to_workingCp_ratio,debtEquity_ratio,quick_ratio,totalShareholderEquity,ROE_%,liquidation_value,liquidation_mktcap_ratio_%,fiscalDateEnding
64,34.833869,18.649518,14.362272,1072000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,60.683977,44.255858,30.664978,2421000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,44.329694,30.500639,22.179601,2255000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,44.432194,28.457675,19.845645,1620000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,47.774979,31.570109,21.926352,1828000000,2.110918,1.850900e+10,NaN,0.859549,2.088110,2.588800e+10,7.061187,2.542200e+10,0.77,2009-06-30
59,41.820267,30.179405,20.742197,2532000000,2.742482,2.004900e+10,NaN,0.501296,2.702938,3.164000e+10,8.002528,3.118700e+10,0.95,2009-09-30
58,40.878658,30.128164,21.539246,3378000000,2.545010,2.023500e+10,NaN,0.507660,2.501031,3.576800e+10,9.444196,3.527400e+10,1.07,2009-12-31
57,41.669753,29.476258,22.772057,3074000000,2.644206,2.010700e+10,NaN,0.450061,2.592035,3.934800e+10,7.812341,3.860500e+10,1.17,2010-03-31
56,39.082803,26.968153,20.719745,3253000000,2.308032,2.042100e+10,NaN,0.501357,2.247694,4.311100e+10,7.545638,4.207900e+10,1.28,2010-06-30
55,36.926707,26.775795,21.176818,4308000000,2.011292,2.095600e+10,NaN,0.573162,1.960573,4.779100e+10,9.014250,4.670800e+10,1.42,2010-09-30


In [175]:
# 90% accurate
annual_income_df[['fiscalDateEnding'] + income_ratio_cols]  

,gross_margin,operating_margin,net_margin,fiscalDateEnding
13,80.162922,38.566673,30.023686,2010-06-30
12,77.729008,38.833050,33.098380,2011-06-30
11,76.221803,29.519960,23.029448,2012-06-30
10,73.814693,34.379375,28.083855,2013-06-30
9,68.816003,31.968261,25.421211,2014-06-30
8,64.695448,19.406925,13.029493,2015-06-30
7,64.038879,28.608728,22.532198,2016-06-30
6,64.522476,30.055607,26.394052,2017-06-30
5,67.156927,32.696648,15.454851,2018-06-30
4,67.154946,34.786024,31.774566,2019-06-30


In [25]:
# 90% accurate
annual_balancesheet_df[['fiscalDateEnding'] + balancesheet_ratio_cols]

,fiscalDateEnding,current_ratio,working_capital,longTermDebt_to_workingCp_ratio,debtEquity_ratio,quick_ratio,totalShareholderEquity
14,2009-06-30,1.822890,22246000000,0.168839,0.968957,1.796368,39558000000
13,2010-06-30,2.129346,29529000000,0.165939,0.864927,2.101044,46175000000
12,2011-06-30,2.603670,46144000000,0.257888,0.904315,2.555988,57083000000
11,2012-06-30,2.602912,52396000000,0.227117,0.827389,2.568129,66363000000
10,2013-06-30,2.711762,64049000000,0.243564,0.804203,2.659967,78944000000
9,2014-06-30,2.504022,68621000000,0.300200,0.919986,2.445721,89784000000
8,2015-06-30,2.473402,73150000000,0.414217,1.178640,2.414950,80083000000
7,2016-06-30,2.352882,80303000000,0.505585,1.461921,2.314959,83090000000
6,2017-06-30,2.918576,106951000000,0.720891,1.853827,2.879451,87711000000
5,2018-06-30,2.900800,111174000000,0.685412,2.129283,2.855287,82718000000


In [38]:
import requests

symbol = 'MSFT'
# COMPANY OVERVIEW
# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}'
r = requests.get(url)
data = r.json()

# pd.DataFrame(data)

for key, value in data.items():
    # print(key)
    if key in ['Sector', 'Industry', 'PEGRatio', 'MarketCapitalization', 'ForwardPE']:
        print(key, value)

Sector TECHNOLOGY
Industry SERVICES-PREPACKAGED SOFTWARE
MarketCapitalization 3017962684000
PEGRatio 2.187
ForwardPE 30.58


In [176]:
# # List of columns to plot
# # columns_to_plot = qtr_income_df.columns[1:]  # Exclude the 'fiscalDateEnding' column

# # Create bar charts for each column
# for column in qtr_income_ratio_cols:
#     fig = px.bar(qtr_income_df, x='fiscalDateEnding', y=column, title=column)
#     fig.show()

In [11]:
symbol = 'MSFT'

# BALANCESHEET
# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={symbol}&apikey={alpha_vantage_api_key}'
r = requests.get(url)
data = r.json()

for key, value in data.items():
    print(key, value)

    if key == 'annualReports':
        annual_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
        annual_balancesheet_df = annual_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

        for column in annual_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
            annual_balancesheet_df[column] = pd.to_numeric(annual_balancesheet_df[column], errors='coerce')

    if key == 'quarterlyReports':
        qtr_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
        qtr_balancesheet_df = qtr_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

        for column in qtr_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
            qtr_balancesheet_df[column] = pd.to_numeric(qtr_balancesheet_df[column], errors='coerce')

symbol MSFT
annualReports [{'fiscalDateEnding': '2023-06-30', 'reportedCurrency': 'USD', 'totalAssets': '411976000000', 'totalCurrentAssets': '184257000000', 'cashAndCashEquivalentsAtCarryingValue': '34704000000', 'cashAndShortTermInvestments': '111262000000', 'inventory': '2500000000', 'currentNetReceivables': '48688000000', 'totalNonCurrentAssets': '232219000000', 'propertyPlantEquipment': '95641000000', 'accumulatedDepreciationAmortizationPPE': '68251000000', 'intangibleAssets': '77252000000', 'intangibleAssetsExcludingGoodwill': '9366000000', 'goodwill': '67886000000', 'investments': '95599000000', 'longTermInvestments': '9879000000', 'shortTermInvestments': '76558000000', 'otherCurrentAssets': '21807000000', 'otherNonCurrentAssets': '30601000000', 'totalLiabilities': '205753000000', 'totalCurrentLiabilities': '104149000000', 'currentAccountsPayable': '18095000000', 'deferredRevenue': '107626000000', 'currentDebt': '5247000000', 'shortTermDebt': '0', 'totalNonCurrentLiabilities': '

In [14]:
qtr_balancesheet_df.tail(5)

,fiscalDateEnding,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,accumulatedDepreciationAmortizationPPE,intangibleAssets,intangibleAssetsExcludingGoodwill,goodwill,investments,longTermInvestments,shortTermInvestments,otherCurrentAssets,otherNonCurrentAssets,totalLiabilities,totalCurrentLiabilities,currentAccountsPayable,deferredRevenue,currentDebt,shortTermDebt,totalNonCurrentLiabilities,capitalLeaseObligations,longTermDebt,currentLongTermDebt,longTermDebtNoncurrent,shortLongTermDebtTotal,otherCurrentLiabilities,otherNonCurrentLiabilities,totalShareholderEquity,treasuryStock,retainedEarnings,commonStock,commonStockSharesOutstanding
4,2023-03-31,380088000000,163889000000,26562000000,104427000000,2877000000,38137000000,220499000000,88132000000,65998000000,77819000000,9879000000,67940000000,9.518600e+10,9415000000,77865000000,19165000000,26954000000,185405000000,85691000000,15305000000,7.920200e+10,6.245000e+09,6245000000,99714000000,1.231200e+10,48210000000,6.245000e+09,41965000000,48210000000,12664000000,17437000000,194683000000,NaN,108234000000,92093000000,7437000000
3,2023-06-30,411976000000,184257000000,34704000000,111262000000,2500000000,48688000000,232219000000,95641000000,68251000000,77252000000,9366000000,67886000000,9.559900e+10,9879000000,76558000000,21807000000,30601000000,205753000000,104149000000,18095000000,1.076260e+11,5.247000e+09,0,101604000000,1.272800e+10,47237000000,5.247000e+09,41990000000,47237000000,14745000000,17981000000,206223000000,NaN,118848000000,93718000000,7432000000
2,2023-09-30,445785000000,207586000000,80452000000,143951000000,3000000000,36953000000,242699000000,102502000000,69486000000,76685000000,8895000000,67790000000,6.875000e+10,11423000000,63499000000,23682000000,32154000000,225071000000,124792000000,19307000000,9.837600e+10,2.955600e+10,25808000000,100279000000,1.348700e+10,45694000000,3.748000e+09,41946000000,71502000000,14475000000,18634000000,220714000000,NaN,132143000000,95508000000,7431000000
1,2023-12-31,470558000000,147393000000,17305000000,81017000000,1615000000,42831000000,327965000000,112308000000,72949000000,148827000000,29896000000,118931000000,7.148200e+10,13367000000,63712000000,21930000000,32265000000,232290000000,121016000000,17695000000,9.206800e+10,2.929100e+10,27041000000,111274000000,1.415500e+10,47178000000,2.250000e+09,44928000000,74219000000,16362000000,20787000000,238268000000,NaN,145737000000,97480000000,7432000000
0,2024-03-31,484275000000,147180000000,19634000000,80021000000,1304000000,44029000000,341895000000,121375000000,74945000000,147991000000,28828000000,119163000000,6.979100e+10,14807000000,60387000000,21826000000,35551000000,231123000000,118525000000,18087000000,8.966600e+10,2.274900e+10,20500000000,112598000000,1.446900e+10,44907000000,2.249000e+09,42658000000,65407000000,18023000000,23271000000,253152000000,NaN,159394000000,99193000000,7433000000


# SP500 sectors

In [38]:
# Function to get the list of S&P 500 companies and their sectors
def get_sp500_companies():
    # Fetch the S&P 500 company symbols and sectors from a reliable source (e.g., Wikipedia)
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    tables = pd.read_html(url)
    
    # Extract the relevant table containing the company symbols and sectors
    sp500_df = tables[0]
    
    # Return the DataFrame containing S&P 500 companies and sectors
    return sp500_df[['Symbol', 'GICS Sector', 'GICS Sub-Industry']]


# Function to create a dictionary of sectors and sub-sectors
def create_sector_subsector_dict(df):
    sector_subsector_dict = {}
    for index, row in df.iterrows():
        sector = row['GICS Sector']
        subsector = row['GICS Sub-Industry']
        if sector not in sector_subsector_dict:
            sector_subsector_dict[sector] = [subsector]
        else:
            sector_subsector_dict[sector].append(subsector)
    return sector_subsector_dict

# Function to filter the S&P 500 companies by sector
def company_sector_list(df, sector):
    return df[df['GICS Sector'] == sector]['Symbol'].tolist()

def company_sub_sector_list(df, sub_sector):
    return df[df['GICS Sub-Industry'] == sub_sector]['Symbol'].tolist()


# Get the list of S&P 500 companies and their sectors
sp500_df  = get_sp500_companies()

sp500_companies_sectors = sp500_df ['GICS Sector'].value_counts().index
sp500_companies_sub_sectors = sp500_df ['GICS Sub-Industry'].value_counts().index

sector_subsector_dict = create_sector_subsector_dict(sp500_df)


# Function to create a DataFrame from the sector_subsector_dict
def create_sector_dataframe():
    # Create a list to store dictionacompany_sector_listries representing each row of data
    data = []
    
    # Filter the DataFrame to get stocks in the specified sector
    for sector in sp500_companies_sectors:
        sector_stocks_list = company_sector_list(sp500_df, sector)

        # Iterate over the stocks in the sector and create a dictionary for each
        for i, ticker in enumerate(sector_stocks_list, start=1):
            # Create a dictionary for the current stock in the sector
            row_data = {'Sector': sector, 'Ticker': ticker}
            # Append the dictionary to the list
            data.append(row_data)
    
    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data)
    return df


sector_subsector_dict = create_sector_subsector_dict(sp500_df)


sector_ticker_df = create_sector_dataframe()
sector_ticker_list = sector_ticker_df[sector_ticker_df['Sector'] == 'Consumer Staples']['Ticker'].tolist()

In [39]:
# Pivot the DataFrame
pivot_sector_ticker_df = sector_ticker_df.groupby('Sector')['Ticker'].apply(list).reset_index()

# Transpose to get sectors as columns
pivot_sector_ticker_df = pivot_sector_ticker_df.set_index('Sector').T

pivot_sector_ticker_df

Sector,Communication Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities
Ticker,"[GOOGL, GOOG, T, CHTR, CMCSA, EA, FOXA, FOX, I...","[ABNB, AMZN, APTV, AZO, BBWI, BBY, BKNG, BWA, ...","[MO, ADM, BF.B, BG, CPB, CHD, CLX, KO, CL, CAG...","[APA, BKR, CVX, COP, CTRA, DVN, FANG, EOG, EQT...","[AFL, ALL, AXP, AIG, AMP, AON, ACGL, AJG, AIZ,...","[ABT, ABBV, A, ALGN, AMGN, BAX, BDX, BIO, TECH...","[MMM, AOS, ALLE, AAL, AME, ADP, AXON, BA, BR, ...","[ACN, ADBE, AMD, AKAM, APH, ADI, ANSS, AAPL, A...","[APD, ALB, AMCR, AVY, BALL, CE, CF, CTVA, DOW,...","[ARE, AMT, AVB, BXP, CPT, CBRE, CSGP, CCI, DLR...","[AES, LNT, AEE, AEP, AWK, ATO, CNP, CMS, ED, C..."


In [15]:
pivot_sector_ticker_df['Financials'].iloc[0]

['AFL',
 'ALL',
 'AXP',
 'AIG',
 'AMP',
 'AON',
 'ACGL',
 'AJG',
 'AIZ',
 'BAC',
 'BK',
 'BRK.B',
 'BLK',
 'BX',
 'BRO',
 'COF',
 'CBOE',
 'SCHW',
 'CB',
 'CINF',
 'C',
 'CFG',
 'CME',
 'CPAY',
 'DFS',
 'EG',
 'FDS',
 'FIS',
 'FITB',
 'FI',
 'BEN',
 'GPN',
 'GL',
 'GS',
 'HIG',
 'HBAN',
 'ICE',
 'IVZ',
 'JKHY',
 'JPM',
 'KEY',
 'KKR',
 'L',
 'MTB',
 'MKTX',
 'MMC',
 'MA',
 'MET',
 'MCO',
 'MS',
 'MSCI',
 'NDAQ',
 'NTRS',
 'PYPL',
 'PNC',
 'PFG',
 'PGR',
 'PRU',
 'RJF',
 'RF',
 'SPGI',
 'STT',
 'SYF',
 'TROW',
 'TRV',
 'TFC',
 'USB',
 'V',
 'WRB',
 'WFC',
 'WTW']

In [16]:
len(pivot_sector_ticker_df['Financials'].iloc[0])

71

## Appendix

In [128]:
# Consumer Staples

ticker_symbols = [
    'MO',
    'ADM',
    # 'BF.B', # DATA ISSUE
    'BG',
    'CPB',
    'CHD',
    'CLX',
    'KO',
    'CL',
    'CAG',
    'STZ', # !!
    'COST',
    'DG',
    'DLTR', # !!
    'EL',
    'GIS',
    'HSY',
    'HRL',
    'K',
    'KVUE',
    'KDP',
    'KMB',
    'KHC',
    # 'KR', # DATA ISSUE
    'LW',
    # 'MKC', # DATA ISSUE
    'TAP',
    'MDLZ',
    'MNST',
    'PEP',
    'PM',
    'PG',
    'SJM',
    'SYY',
    'TGT', # !!
    'TSN', # !!
    # 'WBA', # DATA ISSUE
    'WMT'
]

In [ ]:
# Energy

ticker_symbols = [
'APA',
 'BKR', # !! HIGH PE, LOW PROFITABILITY, HIGH FORECAST,  BUT CANNOT BEAT FORECAST, 总结与TRGP类似, 基本面更稳妥, 盈利预期高, 有待考证
 'CVX',
 'COP',
 'CTRA',
 'DVN',
 'FANG',
 'EOG',
 'EQT',
 'XOM',
 'HAL', # !!!! 与SLB类似
 'HES', # !  HIGH PE, UNDERVALUE, 市场炒作高预期, 2024表现强劲, 已经被超热了
 'KMI',
 'MRO',
 'MPC',
 'OXY',
 'OKE', # !!
 'PSX',
 'SLB', # !!! A BIT HIGH PE, GOOD ROE, MED PROFITABILITY, STEADY GROWTH FORECAST, AND CAN BEAT FORECAST, 总结基本面良好，与HAL类型相似, 市场炒作热度比HAL更高, 值得研究
 'TRGP', # !！ HGIH PE, HIGH ROE, RELATIVE MORE DEBT THAN EQUITY, LOW PROFITABILITY, HIGH FORECAST, BUT CANNOT BEAT FORECAST, 总结基本面一般, 但是盈利预期非常高, 有待考证
 'VLO',
 'WMB'
 ]